In [1]:
# для работы с папками
import os

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для дафреймов
import pandas as pd

In [2]:
file_path = 'C:/00_Projects/NYT/new/'

In [3]:
# вытаскиваю названия файлов в список file_list

file_list = []

for root, dirs, files in os.walk(file_path + 'words/'):
    
    for filename in files:
        file_list += [filename]

In [4]:
print(file_list)

['words_1850_1859.csv', 'words_1860_1869.csv', 'words_1870_1879.csv', 'words_1880_1889.csv', 'words_1890_1899.csv', 'words_1900_1909.csv', 'words_1910_1919.csv', 'words_1920_1929.csv', 'words_1930_1939.csv', 'words_1940_1949.csv', 'words_1950_1959.csv', 'words_1960_1969.csv', 'words_1970_1979.csv', 'words_1980_1989.csv', 'words_1990_1999.csv', 'words_2000_2009.csv', 'words_2010_2019.csv', 'words_2020_2029.csv']


In [35]:
decade_words = pd.DataFrame() # датафрейм для слов декады
year_words = pd.DataFrame() # датафрейм для слов года

for decade_file in tqdm(file_list):
    decade_work = pd.read_csv(file_path + 'words/' + decade_file)

    # топ-100 слов декады
    top_100_for_decade = (decade_work.groupby(['decade', 'word'], as_index=False)
                          .agg({'number': 'sum'})
                          .sort_values('number', ascending=False))
    
    decade_words = decade_words.append(top_100_for_decade) # убрал head на время
    
    
    # топ-100 слов года
    start_year = decade_work.year.min()
    end_year = decade_work.year.max()
    
    for need_year in range(start_year, end_year + 1):
        year_work = decade_work[decade_work['year'] == need_year]        
        
        year_words = year_words.append(year_work.groupby(['decade', 'year', 'word'], as_index=False)
                                       .agg({'number': 'sum'})
                                       .sort_values('number', ascending=False)
                                       .head(100))   

  0%|          | 0/18 [00:00<?, ?it/s]

In [7]:
# записывыю в файлы
decade_words.to_csv(file_path + 'decade_words.csv', encoding='utf-8', index=False)
decade_words.to_excel(file_path + 'decade_words.xlsx', encoding='utf-8', index=False)

year_words.to_csv(file_path + 'year_words.csv', encoding='utf-8', index=False)
year_words.to_excel(file_path + 'year_words.xlsx', encoding='utf-8', index=False)

In [36]:
year_words.shape

(17200, 4)

In [37]:
special_word_list = ['war', 'art', 'death', 'win', 'market', 'force', 'love', 'pandemic', 'world', 'peace']

In [38]:
special_word = pd.DataFrame(special_word_list, columns=['word'])

In [39]:
special_word = special_word.merge(year_words, how='left', on='word')

In [40]:
(special_word.groupby(['word', 'year'], as_index=False)
 .agg({'number': 'sum'})
)

,word,year,number
0,art,2009,3796.0
1,art,2011,3486.0
2,art,2012,2351.0
3,art,2013,1893.0
4,art,2014,2009.0
...,...,...,...
365,world,2018,1047.0
366,world,2019,2478.0
367,world,2020,2589.0
368,world,2021,2549.0


In [42]:
special_word.to_excel(file_path + 'special_word.xlsx', index=False)